# Digit Detection Project-DL

In [23]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch 
import numpy as np

In [24]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [25]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [26]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [27]:
print("Training set size:", len(train_data))
print("Test set size:", len(test_data))

Training set size: 60000
Test set size: 10000


In [28]:
loaders = {
    'train' : DataLoader(train_data,
                        batch_size = 100,
                        shuffle = True,
                        num_workers = 1),
    
    'test' : DataLoader(test_data,
                        batch_size = 100,
                        shuffle = True,
                        num_workers = 1),
}

In [29]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x2b2783e6e90>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x2b2781a77d0>}

# nn Model & Architecture

In [30]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)  
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5) 
        self.conv2_drop = nn.Dropout2d() # Regulazation layer
        self.fc1 = nn.Linear(320, 50)  # Fully connected layer with 64*5*5 input features and 128 output features
        self.fc2 = nn.Linear(50, 10)  # Output layer with 128 input features (from previous layer) and 10 output features

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x , training=self.training) 
        x = self.fc2(x)
         
        return F.softmax(x, dim=1)


In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)  # Assuming Net is your neural network model class

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # set the gradient to zero for each batch
        output = model(data)  # make the prediction
        loss = loss_fn(output, target)  # compare the output and the actual digit and calculate the loss
        loss.backward()
        optimizer.step()  # optimizing by the backpropagation
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)} / {len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')



In [32]:

class ThreeLayerConvNet(nn.Module):
    def __init__(self):
        super(ThreeLayerConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 3 * 3, 128)  # Adjusted the input size for the fully connected layer
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 3 * 3)  # Adjusted the size based on the output size of the last convolutional layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

# Instantiate the ThreeLayerConvNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_model = ThreeLayerConvNet().to(device)


In [33]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()  # Accumulating the test loss
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct} / {len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%)\n')


In [34]:
for epoch in range(1,2):
    train(epoch)
    test()

Train Epoch: 1 [0 / 60000 (0%)]	2.304464
Train Epoch: 1 [2000 / 60000 (3%)]	2.292096
Train Epoch: 1 [4000 / 60000 (7%)]	2.169569
Train Epoch: 1 [6000 / 60000 (10%)]	2.012673
Train Epoch: 1 [8000 / 60000 (13%)]	1.901926
Train Epoch: 1 [10000 / 60000 (17%)]	1.784625
Train Epoch: 1 [12000 / 60000 (20%)]	1.770032
Train Epoch: 1 [14000 / 60000 (23%)]	1.739292
Train Epoch: 1 [16000 / 60000 (27%)]	1.773075
Train Epoch: 1 [18000 / 60000 (30%)]	1.673612
Train Epoch: 1 [20000 / 60000 (33%)]	1.684339
Train Epoch: 1 [22000 / 60000 (37%)]	1.660591
Train Epoch: 1 [24000 / 60000 (40%)]	1.677039
Train Epoch: 1 [26000 / 60000 (43%)]	1.640893
Train Epoch: 1 [28000 / 60000 (47%)]	1.689652
Train Epoch: 1 [30000 / 60000 (50%)]	1.573316
Train Epoch: 1 [32000 / 60000 (53%)]	1.628314
Train Epoch: 1 [34000 / 60000 (57%)]	1.626811
Train Epoch: 1 [36000 / 60000 (60%)]	1.604641
Train Epoch: 1 [38000 / 60000 (63%)]	1.683629
Train Epoch: 1 [40000 / 60000 (67%)]	1.607831
Train Epoch: 1 [42000 / 60000 (70%)]	1.681027

In [35]:
class ConvNet3x3MaxPool(nn.Module):
    def __init__(self):
        super(ConvNet3x3MaxPool, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 3 * 3, 128)  # Adjusted the input size for the fully connected layer
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 3 * 3)  # Adjusted the size based on the output size of the last convolutional layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

new_model = ConvNet3x3MaxPool().to(device)



In [36]:
import matplotlib.pyplot as plt

def train(epoch):
    model.train()
    losses = []  # List to store training losses for each epoch
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())  # Append the training loss for this batch
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)} / {len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')
    return losses





In [37]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    losses = []  # List to store test losses for each epoch
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_fn(output, target)
            test_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            losses.append(loss.item())  # Append the test loss for this batch
    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct} / {len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%)\n')
    return losses

# Training and testing the NewNet model
train_losses = []
test_losses = []

for epoch in range(1, 2):
    train_loss = train(epoch)
    test_loss = test()
    train_losses.extend(train_loss)
    test_losses.extend(test_loss)


Train Epoch: 1 [0 / 60000 (0%)]	1.638490
Train Epoch: 1 [2000 / 60000 (3%)]	1.547747
Train Epoch: 1 [4000 / 60000 (7%)]	1.596798
Train Epoch: 1 [6000 / 60000 (10%)]	1.589628
Train Epoch: 1 [8000 / 60000 (13%)]	1.639228
Train Epoch: 1 [10000 / 60000 (17%)]	1.556213
Train Epoch: 1 [12000 / 60000 (20%)]	1.577391
Train Epoch: 1 [14000 / 60000 (23%)]	1.556094
Train Epoch: 1 [16000 / 60000 (27%)]	1.602403
Train Epoch: 1 [18000 / 60000 (30%)]	1.605165
Train Epoch: 1 [20000 / 60000 (33%)]	1.579773
Train Epoch: 1 [22000 / 60000 (37%)]	1.561525
Train Epoch: 1 [24000 / 60000 (40%)]	1.657276
Train Epoch: 1 [26000 / 60000 (43%)]	1.566274
Train Epoch: 1 [28000 / 60000 (47%)]	1.532547
Train Epoch: 1 [30000 / 60000 (50%)]	1.595719
Train Epoch: 1 [32000 / 60000 (53%)]	1.561953
Train Epoch: 1 [34000 / 60000 (57%)]	1.575946
Train Epoch: 1 [36000 / 60000 (60%)]	1.546252
Train Epoch: 1 [38000 / 60000 (63%)]	1.598880
Train Epoch: 1 [40000 / 60000 (67%)]	1.584369
Train Epoch: 1 [42000 / 60000 (70%)]	1.531976

# Evaluation

In [38]:
new_model.eval()
data, target = next(iter(test_data))
data = data.to(device)
output = new_model(data)
prediction = output.argmax(dim=1, keepdim=True).squeeze().cpu().numpy()
print(f'Prediction: {prediction}')


Prediction: 7


In [39]:
model.eval()
data, target = next(iter(test_data))  
data = data.to(device)
output = model(data)
prediction = output.argmax(dim=1, keepdim=True).squeeze().cpu().numpy()
print(f'Prediction: {prediction}')


Prediction: 7


In [40]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

# Function to train the model
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Function to evaluate the model
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

# Function to conduct hyperparameter experimentation
def hyperparameter_experiment(train_loader, test_loader, lr_values, batch_sizes, epochs, device):
    results = []
    for lr in lr_values:
        for batch_size in batch_sizes:
            for epoch in epochs:
                model = Net().to(device)
                optimizer = optim.Adam(model.parameters(), lr=lr)
                criterion = nn.CrossEntropyLoss()

                train_losses = []
                for _ in range(epoch):
                    train_loss = train(model, train_loader, optimizer, criterion, device)
                    train_losses.append(train_loss)

                test_loss, accuracy = test(model, test_loader, criterion, device)

                result = {
                    'learning_rate': lr,
                    'batch_size': batch_size,
                    'epochs': epoch,
                    'test_loss': test_loss,
                    'accuracy': accuracy
                }
                results.append(result)
                print(f"LR: {lr}, Batch Size: {batch_size}, Epochs: {epoch}, Test Loss: {test_loss}, Accuracy: {accuracy}")

    return results

lr_values = [0.001, 0.01]
batch_sizes = [32,64]
epochs = [5,10]

train_data = datasets.MNIST(root='data', train=True, transform=ToTensor(), download=True)
test_data = datasets.MNIST(root='data', train=False, transform=ToTensor(), download=True)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

results = hyperparameter_experiment(train_loader, test_loader, lr_values, batch_sizes, epochs, device)


LR: 0.001, Batch Size: 32, Epochs: 5, Test Loss: 0.0014946893572807313, Accuracy: 96.69
LR: 0.001, Batch Size: 32, Epochs: 10, Test Loss: 0.0014895165324211122, Accuracy: 97.18
LR: 0.001, Batch Size: 64, Epochs: 5, Test Loss: 0.001492116641998291, Accuracy: 96.87
LR: 0.001, Batch Size: 64, Epochs: 10, Test Loss: 0.0014882129073143006, Accuracy: 97.28
LR: 0.01, Batch Size: 32, Epochs: 5, Test Loss: 0.0017133501887321473, Accuracy: 74.78
LR: 0.01, Batch Size: 32, Epochs: 10, Test Loss: 0.0015731307029724121, Accuracy: 88.81
LR: 0.01, Batch Size: 64, Epochs: 5, Test Loss: 0.0016039026021957398, Accuracy: 85.72
LR: 0.01, Batch Size: 64, Epochs: 10, Test Loss: 0.0016185505032539369, Accuracy: 84.26


In [42]:
print("Learning Rate\tBatch Size\tEpochs\tTest Loss\tAccuracy")
for result in results:
    print(f"{result['learning_rate']}\t\t{result['batch_size']}\t\t{result['epochs']}\t\t{result['test_loss']:.4f}\t\t{result['accuracy']:.2f}%")


Learning Rate	Batch Size	Epochs	Test Loss	Accuracy
0.001		32		5		0.0015		96.69%
0.001		32		10		0.0015		97.18%
0.001		64		5		0.0015		96.87%
0.001		64		10		0.0015		97.28%
0.01		32		5		0.0017		74.78%
0.01		32		10		0.0016		88.81%
0.01		64		5		0.0016		85.72%
0.01		64		10		0.0016		84.26%
